In [1]:
import pandas as pd
from nltk.corpus import stopwords
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
users_df = pd.read_excel("../data/Visitors Preference Dataset.xlsx")
places_df = pd.read_excel("../data/Places Dataset.xlsx")

In [3]:
users_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."


In [4]:
places_df.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


In [5]:
selected_features_user = ['Preferred Activities']
selected_features_places = ['name', 'latest_reviews']

In [6]:
users_df = users_df[selected_features_user]
places_df = places_df[selected_features_places]

In [7]:
users_df.fillna("", inplace=True)
places_df.fillna("", inplace=True)

In [8]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: ' '.join(x.split()))

In [9]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(
    lambda x: eval(x) if isinstance(x, str) and x.strip() != '' else ''
)

In [10]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: " ".join(x))

In [11]:
users_df.head()

,Preferred Activities
0,cycling historical monuments village homestays
1,butterfly watching hot springs wildlife viewing
2,sea cruises themed parks craft workshops
3,fishing hot springs sailing
4,history tours sailing literary tours


In [12]:
places_df['latest_reviews'] = places_df['latest_reviews'].apply(
    lambda x: ''.join([i if i.isalpha() or i.isspace() else '' for i in x])
)

In [13]:
places_df['latest_reviews'] = places_df['latest_reviews'].apply(lambda x: x.lower())
places_df['name'] = places_df['name'].apply(lambda x: x.lower())
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: x.lower())

In [14]:
stop_words = set(stopwords.words('english'))

places_df['latest_reviews'] = places_df['latest_reviews'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)

In [15]:
places_df.head()

,name,latest_reviews
0,arugam bay beach,arugam bay beach surfers paradise spent incred...
1,mirissa beach,mirissa beach truly gem sri lankaãââs southern...
2,weligama beach (surf and stay),weligama beach fantastic spot beginner experie...
3,ahangama,ahangama bit disappointing solo traveler surfi...
4,hikkaduwa beach,hikkaduwa beach delightful escape solo travele...


In [16]:
combined_features_places = places_df['name'] + ' ' + places_df['latest_reviews']
combined_features_users = users_df['Preferred Activities']

In [17]:
combined_features_places[0], combined_features_users[0]

('arugam bay beach arugam bay beach surfers paradise spent incredible days riding waves local surf schools fantastic beginners like atmosphere laidback friendly locals fellow travelers long day surfing sunsets simply magical beach bit crowded especially peak season adds lively vibe canãâât wait return friends unforgettable time arugam bay beach surfing conditions excellent managed catch great waves beach beautiful soft sand clear waters perfect swimming however noticed litter beach bit disappointing overall vibrant nightlife delicious food made definitely worth visit couple looking relaxation arugam bay beach offered perfect blend tranquility excitement enjoyed lazy days lounging beach indulging fresh seafood beachside restaurants surf scene lively easy find quieter spots unwind downside occasional noise nearby parties didnãâât detract much experience lovely getaway visited arugam bay beach family children loved surf lessons found beach bit overcrowded atmosphere vibrant locals warm we

In [19]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_places = tfidf_vectorizer.fit_transform(combined_features_places)
tfidf_matrix_users = tfidf_vectorizer.transform(combined_features_users)

In [20]:
tfidf_matrix_places.shape

(411, 4482)

In [21]:
tfidf_matrix_users.shape

(10000, 4482)

In [22]:
cosine_sim = cosine_similarity(tfidf_matrix_users, tfidf_matrix_places)

In [23]:
cosine_sim.shape

(10000, 411)

In [24]:
cosine_sim[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01628877, 0.02106451,
       0.01991881, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.12725004, 0.01114984,
       0.        , 0.        , 0.01810857, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04849622, 0.        ,
       0.        , 0.        , 0.01611308, 0.        , 0.01601016,
       0.        , 0.        , 0.        , 0.01632363, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [25]:
top_5_places = cosine_sim[0].argsort()[-5:][::-1]

In [26]:
places_original = pd.read_excel("../data/Places Dataset.xlsx")
users_original = pd.read_excel("../data/Visitors Preference Dataset.xlsx")

In [27]:
places_original.iloc[top_5_places]

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
95,Polonnaruwa,7.940338,81.018798,"Polonnaruwa, Sri Lanka",NaN,NaN,['Polonnaruwa felt like a missed opportunity. ...
23,Anuradhapura,8.311352,80.403651,"Anuradhapura, Sri Lanka",NaN,NaN,"['Anuradhapura is rich in history, but I found..."
100,Viharamahadevi Park,6.913391,79.861740,"Colombo, Sri Lanka",4.1,13573.0,['Viharamahadevi Park is a gem in the bustling...
340,Mandathivu Beach,9.603580,80.000259,"Mandaitivu South, Sri Lanka",4.8,129.0,['Mandathivu Beach is a hidden gem! I visited ...
38,Colombo National Museum,6.909999,79.860903,"Colombo , Sri Lanka",4.6,5292.0,['The Colombo National Museum is a treasure tr...


In [28]:
for i in top_5_places:
    print(f"Similarity measure for place {places_original.iloc[i]['name']} is {cosine_sim[0][i]}")

Similarity measure for place Polonnaruwa is 0.13826850195559753
Similarity measure for place Anuradhapura is 0.12725003807823496
Similarity measure for place Viharamahadevi Park is 0.11405149983487055
Similarity measure for place Mandathivu Beach is 0.05162065406762057
Similarity measure for place Colombo National Museum is 0.04849622294158348


In [29]:
users_original.iloc[0]

User ID                                                                               1
Name                                                                     Jennifer Quinn
Email                                                        jennifer.quinn@example.com
Preferred Activities                  ['cycling', 'historical monuments', 'village h...
Bucket list destinations Sri Lanka    ['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell...
Name: 0, dtype: object

In [30]:
# Let's put everything in a function
def get_recommendations(user_id):
    user = users_original.iloc[user_id]
    user_features = user['Preferred Activities'] + ' ' + user['Bucket list destinations Sri Lanka']
    user_features = user_features.lower()
    print(f"User features: {user_features}")
    user_features = ' '.join([word for word in user_features.split() if word not in stop_words])
    user_features = [user_features]
    user_tfidf = tfidf_vectorizer.transform(user_features)
    cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix_places)
    top_5_places = cosine_sim[0].argsort()[-10:][::-1]
    for i in top_5_places:
        print(f"Similarity measure for place {places_original.iloc[i]['name']} is {cosine_sim[0][i]}")
    return places_original.iloc[top_5_places]

In [31]:
get_recommendations(0)

User features: ['cycling', 'historical monuments', 'village homestays'] ['polonnaruwa', 'hatton', 'anuradhapura', 'ella', 'haputale']
Similarity measure for place Haputale is 0.25095414880914707
Similarity measure for place Anuradhapura is 0.22026950906501558
Similarity measure for place Polonnaruwa is 0.21555679987644327
Similarity measure for place Polonnaruwa Ancient City is 0.130695473801295
Similarity measure for place Pandalawa Ella is 0.10683480083621741
Similarity measure for place Watawala Ella is 0.104726295317535
Similarity measure for place Polonnaruwa Sivan Kovil is 0.10405751783972599
Similarity measure for place Madola Ella is 0.1016145093498129
Similarity measure for place Anuradhapura New Town is 0.10127952920262515
Similarity measure for place Dhushan ella is 0.09789782806430625


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
78,Haputale,6.765414,80.952565,"Haputale, Sri Lanka",NaN,NaN,['Haputale is a beautiful location with stunni...
23,Anuradhapura,8.311352,80.403651,"Anuradhapura, Sri Lanka",NaN,NaN,"['Anuradhapura is rich in history, but I found..."
95,Polonnaruwa,7.940338,81.018798,"Polonnaruwa, Sri Lanka",NaN,NaN,['Polonnaruwa felt like a missed opportunity. ...
9,Polonnaruwa Ancient City,7.945942,81.000329,"Polonnaruwa, Sri Lanka",4.3,878.0,['Polonnaruwa Ancient City is a stunning place...
301,Pandalawa Ella,6.737742,80.399637,"Ratnapura, Sri Lanka",3.8,130.0,['Pandalawa Ella is a hidden gem! The scenic v...
299,Watawala Ella,6.628570,80.360297,"Niriella, Sri Lanka",NaN,NaN,"['Watawala Ella has stunning views, but I foun..."
368,Polonnaruwa Sivan Kovil,7.946288,81.001328,"Polonnaruwa, Sri Lanka",4.8,146.0,['Visiting Polonnaruwa Sivan Kovil was a fasci...
303,Madola Ella,6.621315,80.481563,"Ratnapura, Sri Lanka",4.6,27.0,['Madola Ella is a hidden gem! The journey to ...
315,Anuradhapura New Town,8.322765,80.402577,"Anuradhapura, Sri Lanka",4.4,167.0,['Anuradhapura New Town is a fascinating blend...
300,Dhushan ella,6.631715,80.354231,"Palawela, Sri Lanka",4.7,59.0,['Dhushan Ella is truly a hidden gem! The brea...


In [32]:
get_recommendations(1)

User features: ['butterfly watching', 'hot springs', 'wildlife viewing'] ['madunagala hot water spring', 'wilpattu national park', 'wasgamuwa national park', 'kanneliya national rain forest reserve', 'horton plains national park']
Similarity measure for place Wasgamuwa National Park is 0.42217699168633355
Similarity measure for place Wilpattu National Park is 0.3869646190154719
Similarity measure for place Madunagala Hot Water Spring is 0.3573878873082388
Similarity measure for place Horton Plains National Park is 0.3461344609732651
Similarity measure for place Minneriya National Park is 0.34301179991299613
Similarity measure for place Kumana National Park is 0.32978276705147347
Similarity measure for place Angammedilla National Park is 0.3164453425530894
Similarity measure for place Bundala National Park is 0.31123083910751104
Similarity measure for place Bundala National Park is 0.30537855539068426
Similarity measure for place Somawathiya National Park is 0.29754253386817364


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
14,Wasgamuwa National Park,7.753242,80.920774,"Nuwaragala, Sri Lanka",4.3,963.0,['Wasgamuwa National Park exceeded my expectat...
13,Wilpattu National Park,8.458182,80.051831,Sri Lanka,4.7,2708.0,['Wilpattu National Park is a fantastic destin...
124,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.6,1117.0,['Madunagala Hot Water Spring is a hidden gem!...
17,Horton Plains National Park,6.809446,80.802333,Sri Lanka,4.7,8564.0,"[\n""Horton Plains National Park was an incredi..."
15,Minneriya National Park,8.015644,80.844670,Sri Lanka,4.6,1293.0,['Minneriya National Park is a must-visit for ...
18,Kumana National Park,6.572960,81.669253,"Okanda, Sri Lanka",4.5,595.0,['Kumana National Park is an absolute delight ...
269,Angammedilla National Park,7.925445,80.920774,"Diyabeduma, Sri Lanka",4.1,287.0,['Angammedilla National Park is a hidden gem! ...
19,Bundala National Park,6.199086,81.210493,"bundala , Sri Lanka",4.3,654.0,['Bundala National Park is a hidden gem for na...
405,Bundala National Park,6.193822,81.187854,"Weligatta, Sri Lanka",4.3,561.0,['Bundala National Park is a hidden gem! I arr...
271,Somawathiya National Park,8.169256,81.207816,"Gangapahalavila, Sri Lanka",4.3,719.0,['Somawathiya National Park is a hidden gem in...


In [33]:
get_recommendations(2)

User features: ['sea cruises', 'themed parks', 'craft workshops'] ['mirissa beach', 'negombo lagoon', 'batadombalena craft centre', 'jungle beach', 'bentota']
Similarity measure for place Jungle Beach is 0.23940425750008598
Similarity measure for place Mirissa Beach is 0.23878115548554824
Similarity measure for place Negombo Lagoon is 0.22992901042987984
Similarity measure for place Cinnamon Bentota Beach is 0.22470662605329386
Similarity measure for place Mirissa is 0.19672299681987143
Similarity measure for place Negombo is 0.17656068995806073
Similarity measure for place Unawatuna Beach is 0.15706429199616434
Similarity measure for place Ventura Beach is 0.14675559085965212
Similarity measure for place Nilaveli Beach is 0.14618210159899167
Similarity measure for place Hambantota Beach is 0.1459167283640829


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
33,Jungle Beach,6.018694,80.239410,"Jungle Beach, Sri Lanka",4.6,2395.0,['Jungle Beach is a hidden gem! The trek to ge...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
45,Negombo Lagoon,7.158004,79.846480,"Negombo Lagoon, Sri Lanka",4.5,268.0,['Negombo Lagoon was a delightful escape! We t...
66,Cinnamon Bentota Beach,6.424905,79.996875,"Bentota , Sri Lanka",4.9,3929.0,['Cinnamon Bentota Beach was a delightful esca...
46,Mirissa,5.948262,80.471587,"Mirissa, Sri Lanka",NaN,NaN,"['I recently went on a sea cruise in Mirissa, ..."
60,Negombo,7.205521,79.851256,"Negombo, Sri Lanka",NaN,NaN,['Negombo was a bit of a letdown for me. I exp...
6,Unawatuna Beach,6.009686,80.248424,"Unawatuna Beach, Sri Lanka",4.8,1868.0,['Unawatuna Beach is a slice of paradise! The ...
402,Ventura Beach,6.423471,79.995320,"Ventura Beach, Sri Lanka",4.4,502.0,['Ventura Beach is a hidden gem! The golden sa...
404,Nilaveli Beach,8.700307,81.192050,"Nilaveli Beach, Sri Lanka",4.5,1247.0,['Nilaveli Beach is a beautiful escape with st...
201,Hambantota Beach,6.127993,81.127204,"Hambantota, Sri Lanka",4.3,168.0,['Hambantota Beach truly exceeded my expectati...


In [34]:
get_recommendations(3)

User features: ['fishing', 'hot springs', 'sailing'] ['maha oya hot water springs', 'colombo port city', 'negombo lagoon', 'trincomalee harbour', 'kalpitiya']
Similarity measure for place Maha Oya Hot Water Springs is 0.5415929165030327
Similarity measure for place Kanniya Hot Water Springs is 0.37361605788529995
Similarity measure for place Madunagala Hot Water Spring is 0.3461706350997142
Similarity measure for place Madunagala Hot Water Spring is 0.31899562372911183
Similarity measure for place Trincomalee Harbour is 0.2777921233679142
Similarity measure for place Port City Colombo is 0.2584448220691907
Similarity measure for place Negombo Lagoon is 0.2544462471037996
Similarity measure for place Kalpitiya Lagoon is 0.24078702036118818
Similarity measure for place Negombo is 0.22085738923772902
Similarity measure for place Trincomalee is 0.17091420314550565


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
123,Maha Oya Hot Water Springs,7.552181,81.353041,"Maha Oya Hot Water Springs, Sri Lanka",4.3,180.0,['Maha Oya Hot Water Springs is a hidden gem! ...
122,Kanniya Hot Water Springs,8.604498,81.171338,"Trincomalee, Sri Lanka",4.3,2538.0,['Kanniya Hot Water Springs is a delightful sp...
399,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.5,1151.0,['Madunagala Hot Water Spring is a hidden gem!...
124,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.6,1117.0,['Madunagala Hot Water Spring is a hidden gem!...
44,Trincomalee Harbour,8.533333,81.250000,"Trincomalee Harbour, Sri Lanka",4.2,284.0,['Trincomalee Harbour is a stunning natural wo...
47,Port City Colombo,6.937804,79.836843,"Port City Colombo, Sri Lanka",NaN,NaN,"['Port City Colombo has a lot of potential, bu..."
45,Negombo Lagoon,7.158004,79.846480,"Negombo Lagoon, Sri Lanka",4.5,268.0,['Negombo Lagoon was a delightful escape! We t...
74,Kalpitiya Lagoon,8.241272,79.744194,"Kalpitiya Lagoon, Sri Lanka",4.8,130.0,['Kalpitiya Lagoon is a fantastic destination ...
60,Negombo,7.205521,79.851256,"Negombo, Sri Lanka",NaN,NaN,['Negombo was a bit of a letdown for me. I exp...
61,Trincomalee,8.587364,81.215212,"Trincomalee, Sri Lanka",NaN,NaN,"[\n""I had high expectations for Trincomalee, b..."


In [35]:
get_recommendations(4)

User features: ['history tours', 'sailing', 'literary tours'] ['negombo lagoon', 'colombo port city', 'galle dutch fort', 'sigiriya', 'jaffna public library']
Similarity measure for place Jaffna Public Library is 0.3588669983239066
Similarity measure for place Dutch Fort - Jaffna is 0.28462977106583043
Similarity measure for place Port City Colombo is 0.27105941996388255
Similarity measure for place Galle Dutch Fort is 0.2635844334163427
Similarity measure for place Negombo Lagoon is 0.25764445219600984
Similarity measure for place Galle Fort Attractions and Jumpers Sri Lanka is 0.23123144870337575
Similarity measure for place Batticaloa Dutch Fort is 0.21183523203991592
Similarity measure for place Jaffna Lagoon Area is 0.20215521735626207
Similarity measure for place Mannar Dutch Fort is 0.20204388201283052
Similarity measure for place Negombo is 0.2008652892925052


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
53,Jaffna Public Library,9.662128,80.011731,"Jaffna, Sri Lanka",4.9,575.0,['The Jaffna Public Library is a beautiful hom...
211,Dutch Fort - Jaffna,9.662031,80.008425,"Jaffna, Sri Lanka",4.7,3672.0,['The Dutch Fort in Jaffna is a splendid testa...
47,Port City Colombo,6.937804,79.836843,"Port City Colombo, Sri Lanka",NaN,NaN,"['Port City Colombo has a lot of potential, bu..."
8,Galle Dutch Fort,6.030459,80.215021,"Galle 80000, Sri Lanka",4.6,16934.0,"[""Galle Dutch Fort is a stunning blend of hist..."
45,Negombo Lagoon,7.158004,79.846480,"Negombo Lagoon, Sri Lanka",4.5,268.0,['Negombo Lagoon was a delightful escape! We t...
192,Galle Fort Attractions and Jumpers Sri Lanka,6.030464,80.215024,"Galle, Sri Lanka",4.7,1328.0,['Galle Fort is an enchanting blend of history...
165,Batticaloa Dutch Fort,7.711813,81.702124,"Batticaloa, Sri Lanka",4.0,1189.0,['The Batticaloa Dutch Fort is a hidden gem! I...
212,Jaffna Lagoon Area,9.657609,80.004428,"Jaffna-Pannai-Kayts Rd, Sri Lanka",3.9,44.0,['Visiting the Jaffna Lagoon Area was a deligh...
242,Mannar Dutch Fort,8.975856,79.916996,"Mannar, Sri Lanka",4.4,817.0,['Visiting Mannar Dutch Fort was a delightful ...
60,Negombo,7.205521,79.851256,"Negombo, Sri Lanka",NaN,NaN,['Negombo was a bit of a letdown for me. I exp...


In [ ]:
get_recommendations(5)

In [ ]:
get_recommendations(6)